In [1]:
!python -V

Python 3.10.12


In [2]:
import os
import torch
import cv2
import numpy as np
import pandas as pd
from os import walk
from os.path import join
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline
# connect to your google drive
from google.colab import drive
from google.colab.patches import cv2_imshow
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
# clone git repositories in workspace
!git clone https://github.com/WongKinYiu/yolov7

# change directory to yolov7 file
%cd /content/yolov7

fatal: destination path 'yolov7' already exists and is not an empty directory.
/content/yolov7


In [ ]:
'''
unzip the file HW1_material.zip which placed in your google drive ex: /content/gdrive/MyDrive/hw1/HW1_material.zip to /content/YoloV7
'''
!unzip "/content/gdrive/MyDrive/YoloV7/HW1_material.zip" -d "/content/yolov7/HW1_material/"
#!mv '/content/gdrive/MyDrive/YoloV7/best.pt' "/content/yolov7/"

## Task B
### Part1: Load a pretrained model and directly apply it to the `HW1_material/detect/parking_area.png` to detect the object.

In [ ]:
# download COCO starting checkpoint
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt

--2024-03-19 06:30:16--  https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... ^C


In [4]:
# run evaluation
!python detect.py --weights yolov7.pt --conf 0.1 --name parking_area --source HW1_material/detect/parking_area.png  --exist-ok

Namespace(weights=['yolov7.pt'], source='HW1_material/detect/parking_area.png', img_size=640, conf_thres=0.1, iou_thres=0.45, device='', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='parking_area', exist_ok=True, no_trace=False)
YOLOR 🚀 v0.1-128-ga207844 torch 2.2.1+cu121 CPU

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
Model Summary: 306 layers, 36905341 parameters, 6652669 gradients
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

/usr/local/lib/python3.10/dist-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
3 persons, 38 cars, 6 trucks, Done. (1674.3ms) Inferen

In [ ]:
# Show image in /content/yolov7/runs/detect/parking_area/parking_area.png
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img = mpimg.imread('/content/yolov7/runs/detect/parking_area/parking_area.png')
imgplot = plt.imshow(img)
plt.show()

### Part2: Learn to fine tuning yolov7 model

In [ ]:
# download COCO starting checkpoint
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7_training.pt

In [ ]:
# Finetune the Yolov7 model
'''
You can add other parameters by using --params_name params_values ex: --device 0 --epoch 1 --batch-size 1
just do not change parameters that we already give you.
hint: only one instruction
'''
'''
Free to load another YoloV7 model that you can find here: https://github.com/WongKinYiu/yolov7
hint: only one instruction

The best model will save in /content/yolov7/runs/train/yolov7-tiny/weights.
If you run this cell again then the model will save if /content/yolov7/runs/train/yolov7-tiny2/weight and so on.
'''
!python train.py --img-size 360 160 --data HW1_material/hw1.yaml --weights 'yolov7-tiny-custom' --exist-ok --cfg cfg/training/yolov7-tiny.yaml --name yolov7-tiny --hyp data/hyp.scratch.custom.yaml

In [5]:
'''Save model in your google drive place'''
savePath = '/content/yolov7/'
!cp  '/content/yolov7/runs/train/yolov7-tiny/weights/best.pt' $savePath

cp: cannot stat '/content/yolov7/runs/train/yolov7-tiny/weights/best.pt': No such file or directory


In [6]:
def Calculate(AnswerFile,PredFile):

  def loadAns(dataPath):
      pos = os.path.join(dataPath, 'car')
      neg = os.path.join(dataPath, 'non-car')
      Answer = []
      for f in os.listdir(pos):
          if not os.path.isdir(f):
              Answer.append((os.path.join(pos, f), 1))
      for f in os.listdir(neg):
          if not os.path.isdir(f):
              Answer.append((os.path.join(neg, f), 0))
      return Answer

  Answer = loadAns(AnswerFile)

  all_negatives, all_positives = 0, 0
  true_negatives, false_negatives = 0, 0
  true_positives, false_positives = 0, 0

  for x, y in Answer:

      exec_path = "'" + x + "'"
      img_filename = x.split('/')[-1]
      img_filename = img_filename.split('.')[0]
      txtpath = PredFile + img_filename + '.txt'

      if y == 1:
          all_positives += 1
      else:
          all_negatives += 1

      try:
          with open(txtpath, 'r') as f:
              n = len(f.readlines())
          if n > 0:
              prediction = 1
          else:
              prediction = 0
      except:
          prediction = 0

      if prediction == 1:
          if y == 1:
              true_positives += 1
          else:
              false_positives += 1
      else:
          if y == 1:
              false_negatives += 1
          else:
              true_negatives += 1

  correct = true_positives + true_negatives
  print("False Positive Rate: %d/%d (%f)" % (false_positives, all_negatives, false_positives/all_negatives))
  print("False Negative Rate: %d/%d (%f)" % (false_negatives, all_positives, false_negatives/all_positives))
  print("Training Accuracy: %d/%d (%f)" % (correct, len(Answer), correct/len(Answer)))

In [ ]:
# If you don't have detect/train just ignore this remind (rm: cannot remove '/content/yolov7/runs/detect/train': No such file or directory)
#!rm -r '/content/yolov7/runs/detect/train'

# Apply on training data
'''
Run evaluation like cell above
you must use parameter: --source /content/yolov7/HW1_material/train/car --name train --save-txt --exist-ok --weights $savePath
hint: one instruction starting from !python
'''
savePath = '/content/yolov7/'
modelPath = os.path.join(savePath,'best.pt')

!python detect.py  --conf 0.4 --source '/content/yolov7/HW1_material/train/car' --name train --save-txt --exist-ok --weights $modelPath
'''
Run evaluation like cell above
you must use parameter: --source /content/yolov7/HW1_material/train/non-car --name train --save-txt --exist-ok --weights $savePath
hint: one instruction starting from !python
'''
!python detect.py --conf 0.4 --source '/content/yolov7/HW1_material/train/non-car' --name train --save-txt --exist-ok --weights $modelPath



In [20]:
Calculate('/content/yolov7/HW1_material/train/','/content/yolov7/runs/detect/train/labels/')

False Positive Rate: 0/300 (0.000000)
False Negative Rate: 37/300 (0.123333)
Training Accuracy: 563/600 (0.938333)


In [ ]:
# If you don't have detect/test just ignore this remind (rm: cannot remove '/content/yolov7/runs/detect/test': No such file or directory)
#!rm -r '/content/yolov7/runs/detect/test'

# Apply on testing data
'''
Run evaluation like cell above
you must use parameter: --source /content/yolov7/HW1_material/test/car --name test --save-txt --exist-ok --weights $savePath
hint: one instruction starting from !python
'''
modelPath = os.path.join(savePath,'best.pt')
!python detect.py  --conf 0.4 --source '/content/yolov7/HW1_material/test/car' --name test --save-txt --exist-ok --weights $modelPath

'''
Run evaluation like cell above
you must use parameter: --source /content/yolov7/HW1_material/test/non-car --name test --save-txt --exist-ok --weights $savePath
hint: one instruction starting from !python
'''
!python detect.py  --conf 0.4 --source '/content/yolov7/HW1_material/test/non-car' --name test --save-txt --exist-ok --weights $modelPath

In [22]:
Calculate('/content/yolov7/HW1_material/test/','/content/yolov7/runs/detect/test/labels/')

False Positive Rate: 0/300 (0.000000)
False Negative Rate: 30/300 (0.100000)
Training Accuracy: 570/600 (0.950000)


In [ ]:
!cp ./*.pt /content/gdrive/MyDrive/YoloV7